In [0]:
from pyspark.sql.functions import datediff, col, avg, count

# Load the core table from the Silver layer
core_df = spark.table("cve_silver.core")

# 1. Calculate the Disclosure Lag in Days
lag_analysis_df = core_df.withColumn(
    "disclosure_lag_days",
    # Calculates the difference between publication and reservation dates
    datediff(col("date_published"), col("date_reserved"))
).filter(col("disclosure_lag_days").isNotNull() & (col("disclosure_lag_days") >= 0)) # Filter out invalid/future dates

print("📊 Disclosure Lag Analysis (Top 5 Slowest Disclosures):")
lag_analysis_df.select("cve_id", "date_reserved", "date_published", "disclosure_lag_days") \
    .orderBy(col("disclosure_lag_days").desc()) \
    .limit(5).display()

# 2. Get Distribution Statistics (Required Output)
print("\n📊 Summary Statistics for Disclosure Lag:")
lag_analysis_df.select("disclosure_lag_days").describe().display()

# 3. Calculate Average Lag for Key Insight
avg_lag = lag_analysis_df.agg(avg(col("disclosure_lag_days")).alias("Average_Lag_Days")).collect()[0]["Average_Lag_Days"]
print(f"🔑 Key Insight: The average disclosure lag for 2024 CVEs is: {avg_lag:.2f} days.")

📊 Disclosure Lag Analysis (Top 5 Slowest Disclosures):


cve_id,date_reserved,date_published,disclosure_lag_days
CVE-2024-21635,2023-12-29T03:00:44.956Z,2025-11-14T14:11:38.230Z,686
CVE-2024-0028,2023-11-16T22:58:45.676Z,2025-09-05T16:10:01.094Z,659
CVE-2024-25621,2024-02-08T22:26:33.511Z,2025-11-06T18:36:21.566Z,637
CVE-2024-21927,2024-01-03T16:43:09.233Z,2025-09-23T21:33:54.121Z,629
CVE-2024-21935,2024-01-03T16:43:14.976Z,2025-09-23T21:38:22.057Z,629



📊 Summary Statistics for Disclosure Lag:


summary,disclosure_lag_days
count,38320
mean,50.82562630480167
stddev,75.92830749799869
min,0
max,686


🔑 Key Insight: The average disclosure lag for 2024 CVEs is: 50.83 days.


In [0]:
# Use the core table directly for severity analysis
core_df = spark.table("cve_silver.core")

print("📊 CVSS Severity Distribution (Risk Bucketing):")
risk_distribution_df = core_df.groupBy("cvss_severity") \
    .agg(
        count("*").alias("count"),
        # Calculate the percentage of total CVEs in each bucket
        (count("*") * 100 / core_df.count()).alias("percentage")
    ) \
    .orderBy(col("count").desc())

risk_distribution_df.display()

# 🔑 Key Insight:
critical_count = risk_distribution_df.filter(col("cvss_severity") == "CRITICAL").select("count").collect()
if critical_count:
    print(f"🔑 Key Insight: There are {critical_count[0]['count']:,} CRITICAL severity vulnerabilities in the dataset.")

📊 CVSS Severity Distribution (Risk Bucketing):


cvss_severity,count,percentage
null,16555,42.71927334657962
MEDIUM,11795,30.436353314582096
HIGH,7588,19.58041958041958
CRITICAL,1788,4.613836348153692
LOW,1015,2.6191520656465306
NONE,12,0.03096534461848115


🔑 Key Insight: There are 1,788 CRITICAL severity vulnerabilities in the dataset.


In [0]:
# Load the affected table (the one row per vendor/product combination)
affected_df = spark.table("cve_silver.affected")

# Group by vendor and count total CVE IDs
vendor_ranking_df = affected_df.groupBy("vendor") \
    .agg(count("cve_id").alias("total_cves")) \
    .orderBy(col("total_cves").desc()) \
    .filter(col("total_cves") > 10) # Filter out noise (vendors with very few CVEs)

print("📊 Top 10 Affected Vendors by Total CVE Count:")
vendor_ranking_df.limit(10).display()

# 🔑 Key Insight:
top_vendor = vendor_ranking_df.collect()[0]
print(f"🔑 Key Insight: The top vendor affected by the highest number of 2024 CVEs is {top_vendor['vendor']} with {top_vendor['total_cves']:,} total affected products/versions.")

📊 Top 10 Affected Vendors by Total CVE Count:


vendor,total_cves
Microsoft,13161
n/a,6591
Linux,6152
"Brother Industries, Ltd",4427
Red Hat,3913
Siemens,2545
Apple,1692
Unknown,1092
Lenovo,929
Adobe,751


🔑 Key Insight: The top vendor affected by the highest number of 2024 CVEs is Microsoft with 13,161 total affected products/versions.
